# 유튜브 웹크롤링을 통한 지역 여행 관심도 수집

- 유튜브는 여행지 탐색 시 자주 활용되는 콘텐츠 플랫폼 중 하나.
- "지역명 + 여행" 키워드로 검색하여 다음과 같은 지표 수집 가능:
    - 검색 결과 영상 수, 상위 영상의 평균 조회수, 최근 업로드 영상 비율
- 이 중 **‘상위 20개 영상의 평균 조회수’**를 대표 지표로 선택
- 선택 이유:
    - 영상 수는 단순 노출 수에 그칠 수 있음
    - 최신 업로드 영상은 사용자 소비와 직접 연결되지 않음 (과거 영상도 지속 시청됨)
    - 조회수는 실제 소비된 콘텐츠의 양을 의미 → 실질적인 관심과 선호를 반영하는 지표로 적합

# 유튜브 검색 키워드 구성

In [2]:
import pandas as pd
# df3 불러오기
df3 = pd.read_csv('지역별_관심도_최종.csv')

In [3]:
df3

,지역,지역별_관심도
0,서울,3.057270
1,부산,3.121910
2,충북,2.802088
3,전북,2.833871
4,강원,3.277752
5,제주,3.062183
6,경기,2.928911
7,인천,2.928911
8,충남,2.918471
9,대전,2.918471


In [4]:
# 지역 리스트 추출
region_list = df3['지역'].tolist()

# '여행' 붙인 검색 키워드 리스트 생성
search_keywords = [region + ' 여행' for region in region_list]

In [5]:
search_keywords

['서울 여행',
 '부산 여행',
 '충북 여행',
 '전북 여행',
 '강원 여행',
 '제주 여행',
 '경기 여행',
 '인천 여행',
 '충남 여행',
 '대전 여행',
 '경남 여행',
 '울산 여행',
 '경북 여행',
 '대구 여행',
 '전남 여행',
 '광주 여행',
 '세종 여행']

# 웹크롤링 코드 구현 과정 (클릭 단계별 세분화 및 문제 해결)

In [193]:
# 유튜브 사이트 열기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
##
'''크롬 브라우저 동작방식 설정'''
# options.add_argument('--headless') # 활성화 하면 크롬창 안보임
options.add_argument("--start-maximized") # 브라우저 크기 최대
options.add_experimental_option("detach", True) # 프로그램이 종료되도 창 유지
options.add_argument("--disable-blink-features=AutomationControlled")
#options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")
##

driver = webdriver.Chrome(options=options)
url = 'https://www.youtube.com/'

driver.get(url)

In [194]:
# 검색창 클릭
driver.find_element(By.XPATH , value = '//*[@id="center"]/yt-searchbox/div[1]/form/input').click()

In [ ]:
# 검색창에 키워드 입력 -> '세종 여행' 예시 -> 추후에 지역 순차적으로 넣는 것으로 코드 수정 예정
driver.find_element(By.XPATH , value = '//*[@id="center"]/yt-searchbox/div[1]/form/input').send_keys('세종 여행')

In [198]:
# 검색버튼 클릭
driver.find_element(By.XPATH , value = '//*[@id="center"]/yt-searchbox/button').click()

- 웹크롤링을 위한 필터 기준 정하기
- 정렬기준: 조회수: 실질적인 콘텐츠 소비 강도와 대중의 관심도를 가장 잘 반영
- 구분: 동영상: 채널, 재생목록 등은 제외하고 실제로 소비되는 개별 영상만 수집
- 길이: 4~20분: 여행 콘텐츠의 핵심 정보를 담기에 적절한 길이. 너무 짧거나 긴 영상은 제외

In [203]:
# 필터 클릭
driver.find_element(By.XPATH , value = '//*[@id="filter-button"]/ytd-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()

In [200]:
# 구분: 동영상 클릭
driver.find_element(By.XPATH, "//yt-formatted-string[text()='동영상']").click()

In [202]:
# 길이: 4~20분
driver.find_element(By.XPATH, "//yt-formatted-string[text()='4~20분']").click()

In [204]:
# 정렬기준: 조회수 클릭
driver.find_element(By.XPATH, "//yt-formatted-string[text()='조회수']").click()

In [205]:
html = driver.page_source # Selenium WebDriver에서 현재 로드된 페이지의 HTML 소스를 문자열로 가져옵
soup = BeautifulSoup(html, 'html.parser') # 가져온 HTML을 BeautifulSoup을 사용하여 파싱

In [ ]:
# 조회수 수집 코드 -> 년도도 포함되어 있어서 '조회수'가 있는 것만 수집하도록 변경
조회수 = [i.text for i in soup.find_all('span', 'inline-metadata-item style-scope ytd-video-meta-block')]
조회수

['조회수 836만회',
 '2년 전',
 '조회수 367만회',
 '2년 전',
 '조회수 128만회',
 '1년 전',
 '조회수 128만회',
 '1년 전',
 '조회수 117만회',
 '2년 전',
 '조회수 69만회',
 '2년 전',
 '조회수 65만회',
 '2년 전',
 '조회수 63만회',
 '1년 전',
 '조회수 57만회',
 '1년 전',
 '조회수 55만회',
 '1개월 전',
 '조회수 51만회',
 '2년 전',
 '조회수 48만회',
 '1년 전',
 '조회수 39만회',
 '4년 전',
 '조회수 33만회',
 '9일 전',
 '조회수 28만회',
 '2년 전',
 '조회수 26만회',
 '4개월 전',
 '조회수 26만회',
 '1년 전',
 '조회수 25만회',
 '3년 전',
 '조회수 23만회',
 '10개월 전',
 '조회수 19만회',
 '2년 전']

In [ ]:
# 조회수 수집 코드
조회수 = [i.text for i in soup.find_all('span', 'inline-metadata-item style-scope ytd-video-meta-block') if '조회수' in i.text]
조회수

['조회수 836만회',
 '조회수 367만회',
 '조회수 128만회',
 '조회수 128만회',
 '조회수 117만회',
 '조회수 69만회',
 '조회수 65만회',
 '조회수 63만회',
 '조회수 57만회',
 '조회수 55만회',
 '조회수 51만회',
 '조회수 48만회',
 '조회수 39만회',
 '조회수 33만회',
 '조회수 28만회',
 '조회수 26만회',
 '조회수 26만회',
 '조회수 25만회',
 '조회수 23만회',
 '조회수 19만회']

In [ ]:
# 1. soup 객체에서 조회수 텍스트 20개 추출
조회수 = [i.text for i in soup.find_all('span', 'inline-metadata-item style-scope ytd-video-meta-block') 
         if '조회수' in i.text][:20]

# 2. "조회수 XXX만회" → 숫자만 추출 후 * 10,000
조회수_숫자 = [int(float(s.replace('조회수', '').replace('만회', '').strip()) * 10000) for s in 조회수]

# 3. 평균 조회수 계산
평균 = sum(조회수_숫자) / len(조회수_숫자)

In [186]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# ✅ 필터 버튼 클릭 함수
def apply_filter(driver, text):
    wait = WebDriverWait(driver, 10)
    filter_button_xpath = '//*[@id="filter-button"]/ytd-button-renderer/yt-button-shape/button'
    wait.until(EC.element_to_be_clickable((By.XPATH, filter_button_xpath))).click()
    filter_option_xpath = f"//yt-formatted-string[text()='{text}']"
    wait.until(EC.element_to_be_clickable((By.XPATH, filter_option_xpath))).click()
    time.sleep(1)

# ✅ 조회수 평균 계산 함수 (영상 순서 정확히 유지)
def get_avg_views(soup, top_n=20):
    videos = soup.find_all('ytd-video-renderer')[:top_n]
    views = []

    for video in videos:
        view_span = video.find('span', string=lambda s: s and '조회수' in s and '만회' in s)
        if view_span:
            match = re.search(r'[\d,.]+', view_span.text)
            if match:
                num = match.group().replace(',', '')
                try:
                    views.append(int(float(num) * 10000))
                except ValueError:
                    continue

    return int(sum(views) / len(views)) if views else 0

# ✅ YouTube 실행 및 데이터 수집 함수
def collect_youtube_view_data(search_keywords):
    # 드라이버 세팅
    options = Options()
    options.add_argument("--start-maximized")
    options.add_experimental_option("detach", True)
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(options=options)

    results = {}

    for keyword in search_keywords:
        driver.get("https://www.youtube.com/")
        time.sleep(2)

        # 검색창 입력
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="center"]/yt-searchbox/div[1]/form/input'))
        )
        search_box.clear()
        search_box.send_keys(keyword)
        search_box.send_keys(Keys.ENTER)
        time.sleep(3)

        # 필터 순차 적용
        for f in ['동영상', '4~20분', '조회수']:
            apply_filter(driver, f)

        # soup 생성 후 평균 조회수 계산
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        avg = get_avg_views(soup)

        results[keyword.replace(" 여행", "")] = avg  # 지역명만 저장
        print(f"{keyword} ▶ 평균 조회수: {avg:,}회")
        time.sleep(2)

    driver.quit()
    return results

# ✅ 실행: 키워드 목록 정의
search_keywords

# ✅ 수집 실행
results = collect_youtube_view_data(search_keywords)

# ✅ Pandas DataFrame 생성
df = pd.DataFrame(list(results.items()), columns=['지역', '유튜브_상위20_평균조회수'])
df = df.sort_values(by='유튜브_상위20_평균조회수', ascending=False).reset_index(drop=True)

# ✅ 결과 출력
print("\n📊 지역별 유튜브 평균 조회수 (상위 20개 기준)")
print(df)

# ✅ CSV 저장 
df.to_csv('지역별_유튜브_평균조회수.csv', index=False)


서울 여행 ▶ 평균 조회수: 807,000회
부산 여행 ▶ 평균 조회수: 1,189,000회
충북 여행 ▶ 평균 조회수: 403,333회
전북 여행 ▶ 평균 조회수: 228,100회
강원 여행 ▶ 평균 조회수: 622,500회
제주 여행 ▶ 평균 조회수: 1,969,500회
경기 여행 ▶ 평균 조회수: 408,000회
인천 여행 ▶ 평균 조회수: 311,400회
충남 여행 ▶ 평균 조회수: 189,900회
대전 여행 ▶ 평균 조회수: 192,850회
경남 여행 ▶ 평균 조회수: 484,500회
울산 여행 ▶ 평균 조회수: 200,750회
경북 여행 ▶ 평균 조회수: 681,000회
대구 여행 ▶ 평균 조회수: 237,950회
전남 여행 ▶ 평균 조회수: 682,050회
광주 여행 ▶ 평균 조회수: 376,450회
세종 여행 ▶ 평균 조회수: 1,101,500회

📊 지역별 유튜브 평균 조회수 (상위 20개 기준)
    지역  유튜브_상위20_평균조회수
0   제주         1969500
1   부산         1189000
2   세종         1101500
3   서울          807000
4   전남          682050
5   경북          681000
6   강원          622500
7   경남          484500
8   경기          408000
9   충북          403333
10  광주          376450
11  인천          311400
12  대구          237950
13  전북          228100
14  울산          200750
15  대전          192850
16  충남          189900


In [187]:
df

,지역,유튜브_상위20_평균조회수
0,제주,1969500
1,부산,1189000
2,세종,1101500
3,서울,807000
4,전남,682050
5,경북,681000
6,강원,622500
7,경남,484500
8,경기,408000
9,충북,403333


- 세종 지역의 평균 조회수가 비정상적으로 높아 직접 영상 확인을 진행
- 확인 결과, 세종 관련 여행 정보가 부족해 여행과 무관한 콘텐츠가 노출된 문제 발견
- 조회수 순 정렬만 적용할 경우, 주제와 무관한 영상도 상위에 노출되어 수집됨
- → 영상 제목에 ‘지역명’과 ‘여행’이 모두 포함된 경우에만 수집하도록 기준 변경

In [213]:
제목 = [i.text.strip() for i in soup.find_all('a', id='video-title') if i.text.strip()]
제목

['외국보다 훨씬 더 멋진 대한민국 여행지 5곳 | 사진보다 실제로 가보면 더 좋은 곳',
 '꼭 가봐야 할 환상적인  국내여행지  BEST 8 amazing tourist attractions in South Korea  인스타핫플 8곳',
 '해외보다 훨씬 더 아름다운 국내여행지 7곳 소개해드릴게요. 죽기 전에 꼭 가봐야 하는 명승지로만 선정했으니 꼭 다녀와 보시기 바랍니다.',
 '충주보틀의 세금여행 【대만1】',
 '해외에 온 것 같은 국내 여행지 Top 6 The  most beautiful and mysterious forest in South Korea  & Relaxing Music',
 '아름다운 아산 여행ㅣ꽉 찬 당일치기ㅣ여행지와 맛집 추천',
 '천상의 정원, 수생식물 학습원 / 충북 옥천 | 유럽풍의 멋진 건축물과 세상에서 제일 작은 교회당, 호수위의 카페 등이 대청호와 어우러진 풍경',
 '하루 1천명이 찾아오는 꼭 가보고 싶은 정원 #여행 #카페 #정원가꾸기',
 '퇴근하고 혼자 성심당간김에 대전여행 브이로그',
 '폴란드 대원 방문기, 남극에서 처음 접한 한식 | 너무 부러워 [남극 🇦🇶 27]',
 '충북 보은ㅣ숙박 + 수영장  + 저녁 + 아침은 물론 단돈 2만원으로 무제한 수육과 공짜로 산양삼까지 먹는 산꼭대기 민박집',
 '주말 3인 55,000원, 5인 75,000원 가성비 대박 숲속 독채',
 '세종대왕이 태종릉에서 옮겨간 까닭은 (시간여행 역사속으로) [역사실험] KBS  2004.12.22 방송',
 "'가족여행 성지'라고 하길래 가봤습니다.",
 '나 혼자 대전 여행 당일치기vlog🍞성심당에서 빵 사고 진로집에서 두부두루치기 • 비빔칼국수에 수육 한 잔, 소제동 카페거리에서 한 잔, 와인도 한 잔, 케이크도 한잔하는 브이로그',
 '형들이랑 옥천여행 먹방투어!!(배말랭 제2의 고향)',
 '신비롭고 독특한 풍광을 가진 대한민국 여행지 6곳 | 좋은 여행지 소개 | Korea Travel Brochure',
 '옥천여행

In [215]:
titles = [i.text.strip() for i in soup.find_all('a', id='video-title') if i.text.strip()]
filtered_titles = [title for title in titles if '세종' in title and '여행' in title]

print("📌 세종 + 여행 포함 제목:")
for t in filtered_titles:
    print("-", t)


📌 세종 + 여행 포함 제목:
- 세종대왕이 태종릉에서 옮겨간 까닭은 (시간여행 역사속으로) [역사실험] KBS  2004.12.22 방송


In [216]:
# 제목과 조회수 리스트 추출
titles = [i.text.strip() for i in soup.find_all('a', id='video-title') if i.text.strip()]
views_raw = [i.text.strip() for i in soup.find_all('span', class_='inline-metadata-item style-scope ytd-video-meta-block') if '조회수' in i.text]

# 제목과 조회수를 짝지어 묶기
title_view_pairs = list(zip(titles, views_raw))

# '세종' + '여행' 포함된 제목만 필터링
filtered_views = [view for title, view in title_view_pairs if '세종' in title and '여행' in title]

# 숫자 변환
views_num = [
    int(float(view.replace('조회수', '').replace('만회', '').replace(',', '').strip()) * 10000)
    for view in filtered_views
    if '만회' in view
]

# 평균 조회수 계산
avg_view = int(sum(views_num) / len(views_num)) if views_num else 0

# 출력
print("📌 세종 + 여행 포함 영상 조회수:", views_num)
print(f"📊 평균 조회수: {avg_view:,}회")


📌 세종 + 여행 포함 영상 조회수: [390000]
📊 평균 조회수: 390,000회


- 유튜브는 기본적으로 첫 페이지에 약 20개의 영상만 노출되며, 세종처럼 콘텐츠가 적은 지역은 영상이 1개만 수집되는 문제 발생 
- → 조건에 맞는 영상이 20개 수집될 때까지 스크롤을 반복하여 데이터 확보

In [217]:
# ▶ 유튜브 검색 후 충분한 영상 로딩
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# 유튜브 접속 및 검색
driver.get("https://www.youtube.com/")
time.sleep(2)

search_box = driver.find_element(By.NAME, 'search_query')
search_box.send_keys("세종 여행")
search_box.send_keys(Keys.ENTER)
time.sleep(3)

# 스크롤 반복 (영상 로딩)
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.documentElement.scrollHeight")

for _ in range(5):  # 더 많이 보고 싶으면 횟수 늘리기
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


# 웹크롤링 진행

In [10]:
search_keywords

['서울 여행',
 '부산 여행',
 '충북 여행',
 '전북 여행',
 '강원 여행',
 '제주 여행',
 '경기 여행',
 '인천 여행',
 '충남 여행',
 '대전 여행',
 '경남 여행',
 '울산 여행',
 '경북 여행',
 '대구 여행',
 '전남 여행',
 '광주 여행',
 '세종 여행']

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# ✅ 조회수 문자열 → 숫자 변환
def parse_view_count(text):
    match = re.search(r'([\d.,]+)([만천]?)회', text)
    if not match:
        return 0
    num, unit = match.groups()
    num = float(num.replace(',', ''))
    if unit == '만':
        return int(num * 10_000)
    elif unit == '천':
        return int(num * 1_000)
    else:
        return int(num)

# ✅ 제목 조건 확인 ('지역'과 '여행' 포함 여부)
def is_valid_title(title, region):
    return '여행' in title and region in title

# ✅ 필터 클릭 함수
def apply_filter(driver, text):
    wait = WebDriverWait(driver, 10)
    filter_button_xpath = '//*[@id="filter-button"]/ytd-button-renderer/yt-button-shape/button'
    wait.until(EC.element_to_be_clickable((By.XPATH, filter_button_xpath))).click()
    filter_option_xpath = f"//yt-formatted-string[text()='{text}']"
    wait.until(EC.element_to_be_clickable((By.XPATH, filter_option_xpath))).click()
    time.sleep(1)

# ✅ 조회수 수집
def get_filtered_avg_views(driver, region, top_n=20):
    collected = []
    seen_titles = set()
    SCROLL_PAUSE_SEC = 2

    while len(collected) < top_n:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        videos = soup.find_all('ytd-video-renderer')

        for video in videos:
            if len(collected) >= top_n:
                break
            title_tag = video.find('a', id='video-title')
            view_tag = video.find('span', string=lambda s: s and '조회수' in s)

            if title_tag and view_tag:
                title = title_tag.text.strip()
                if is_valid_title(title, region) and title not in seen_titles:
                    seen_titles.add(title)
                    views = parse_view_count(view_tag.text)
                    collected.append((title, views))
                    print(f"- {title} ▶ {views:,}회")

        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)

    view_list = [v[1] for v in collected]
    return int(sum(view_list) / len(view_list)) if view_list else 0

# ✅ 전체 실행
def collect_youtube_view_data(search_keywords):
    options = Options()
    options.add_argument("--start-maximized")
    options.add_experimental_option("detach", True)
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(options=options)

    results = {}

    for keyword in search_keywords:
        driver.get("https://www.youtube.com/")
        time.sleep(2)

        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="center"]/yt-searchbox/div[1]/form/input'))
        )
        search_box.clear()
        search_box.send_keys(keyword)
        search_box.send_keys(Keys.ENTER)
        time.sleep(3)

        for f in ['동영상', '4~20분', '조회수']:
            apply_filter(driver, f)

        region_name = keyword.replace(" 여행", "")
        avg = get_filtered_avg_views(driver, region=region_name, top_n=20)
        results[region_name] = avg
        print(f"✅ {region_name} ▶ 평균 조회수: {avg:,}회\n")

    driver.quit()
    return results

# ✅ 키워드 입력 (예시)
search_keywords 
# ✅ 실행
results = collect_youtube_view_data(search_keywords)

# ✅ 결과 출력 및 저장
df = pd.DataFrame(list(results.items()), columns=['지역', '유튜브_상위20_평균조회수'])
df = df.sort_values(by='유튜브_상위20_평균조회수', ascending=False).reset_index(drop=True)

print("\n📊 지역별 유튜브 평균 조회수 (상위 20개 기준)")
print(df)

df.to_csv("지역별_유튜브_평균조회수.csv", index=False)


- 24시간동안 서울 여행 VS 부산 여행!! 어디가 더 재밌을까?! ㅣ파뿌리 ▶ 3,670,000회
- 여기가 서울이라고? 서울토박이도 잘 모르는 숨겨진 서울 여행지 9곳만 꼽아서 소개해 드릴게요. 참고하셔서 즐거운 여행 하시기 바랍니다. ▶ 2,100,000회
- 아오니의 혼자 서울 여행 브이로그 (feat.닌텐도 팝업) ▶ 1,060,000회
- (SUB) 경복궁, 63빌딩은 이제 그만! 트렌디한 서울 데이트 코스 베스트 10 | 국내여행, 서울 가볼만한 곳, 서울 여행, 서울 놀거리, 서울 카페, 성수동 데이트 ▶ 980,000회
- 서울 여행자들을 위한 24시간 공짜식당이 있는 호텔후기 ▶ 970,000회
- 계단이 '단 1개도 없는' 서울 도심속 4.5km 나무 데크길!! | 걷기 초보자도 쉽게 완주할 수 있는 트레킹 코스 | 대중교통 당일치기 여행 | 지하철 여행 | 걷는 여행 ▶ 750,000회
- 아오니의 혼자 서울여행 브이로그2 (feat.동숲 아쿠아리움) ▶ 680,000회
- 서울 여행자들을 위한 최신식 가성비 끝판왕 캡슐호텔 후기 ▶ 540,000회
- 서울 경기도 아이들과 가볼만한 곳 여행 실내 위주로 (방학) ▶ 530,000회
- 부암동 예술가들이 사랑하는 산골 핫플? feat. 청운동｜부암동맛집 서울여행 서울데이트 ▶ 430,000회
- 솔로라서 더 좋은 서울 숨은 여행지! ...있을까? ▶ 380,000회
- 북촌 안국역 서울여행 끝판왕? 제발 이렇게만 즐기세요｜서울여행 서울데이트 ▶ 350,000회
- 겨울에도 따뜻한 서울 실내 여행지 BEST 6 (전부 무료) / 서울 가볼만한 곳, 당일치기 나들이, 데이트 코스, 겨울여행지 ▶ 310,000회
- 서촌 정복하기! 몰라서 못 가는 옥인길, 수성동 계곡, 핫플 맛집 ｜서울여행 서울데이트 ▶ 310,000회
- [&DAY] 먹다 지쳐버린 서울 여행 VLOG | 광장시장 뿌시기 | 북촌 한옥 마을 | 닭한마리 - &TEAM ▶ 280,000회
- 정동길 덕수궁 유명한 이유가 있었네

In [ ]:
import pandas as pd

YouTube = pd.read_csv('지역별_유튜브_평균조회수_0730.csv')

In [13]:
YouTube

,지역,유튜브_상위20_평균조회수
0,제주,1606000
1,서울,725000
2,부산,673500
3,강원,403615
4,경북,321600
5,인천,264000
6,경기,246000
7,광주,239400
8,울산,200500
9,대전,155050
